In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.stats import norm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from math import exp, log, sqrt

In [19]:
file_path = "/Users/pranaykumar/Documents/GitHub/prosperity-3/round3/data/prices_round_3_day_0.csv"
df = pd.read_csv(file_path, delimiter=';')
Strike10000 = df[df['product'] == "VOLCANIC_ROCK_VOUCHER_10000"]
rock = df[df['product'] == "VOLCANIC_ROCK"]

In [20]:
def bs_price(S, K, T, r, sigma, option_type='call'):
    """Black-Scholes price for a European call or put (no dividends)."""
    if T <= 0:
        # At expiration, option value is its intrinsic value
        if option_type == 'call':
            return max(S - K, 0)
        else:
            return max(K - S, 0)
    # Black-Scholes d1 and d2
    d1 = (log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    # CDF of standard normal
    N = lambda x: 0.5 * (1 + math.erf(x / math.sqrt(2)))
    if option_type == 'call':
        return S * N(d1) - K * exp(-r*T) * N(d2)
    else:  # European put
        return K * exp(-r*T) * N(-d2) - S * N(-d1)

def implied_volatility(price, S, K, T, r=0.0, option_type='call'):
    """Solve for implied volatility given an option price via Brent's method."""
    # Basic sanity checks
    if price <= 0 or S <= 0 or K <= 0:
        return float('nan')
    if T <= 0:
        # If at or past expiration, implied vol is not defined (return NaN)
        return float('nan')
    # Compute intrinsic values for bounds
    intrinsic = max(S - K*exp(-r*T), 0) if option_type == 'call' else max(K*exp(-r*T) - S, 0)
    # Arbitrage bounds check
    if option_type == 'call':
        if price < intrinsic or price > S:
            return float('nan')
    else:  # put
        if price < intrinsic or price > K*exp(-r*T):
            return float('nan')
    # Define the target function f(sigma) = BS_price(sigma) - market_price
    def objective(sigma):
        return bs_price(S, K, T, r, sigma, option_type) - price
    # Handle near-intrinsic case: if price is exactly intrinsic, vol ~ 0
    if abs(price - intrinsic) < 1e-12:
        return 0.0
    # Use Brent's method to find volatility root, with a broad bracket
    try:
        iv = optimize.brentq(objective, 1e-10, 5.0, maxiter=1000)
    except (ValueError, RuntimeError):
        # Retry with an even wider bracket if needed
        try:
            iv = optimize.brentq(objective, 1e-10, 10.0, maxiter=1000)
        except Exception:
            iv = float('nan')
    return iv

In [21]:
merged = pd.merge(Strike10000[['timestamp', 'mid_price']], rock[['timestamp', 'mid_price']],
                  on='timestamp', suffixes=('_option', '_underlying'))

# Add Black-Scholes inputs
K = 10000
T = 10 / 365  # or use 5/252 for trading days
r = 0.05

# Compute implied volatility
merged['implied_vol'] = merged.apply(
    lambda row: implied_volatility(row['mid_price_option'], row['mid_price_underlying'], K, T, r),
    axis=1)

# Check the result
merged[['timestamp', 'mid_price_underlying', 'mid_price_option', 'implied_vol']].head()

,timestamp,mid_price_underlying,mid_price_option,implied_vol
0,0,10503.0,505.5,NaN
1,100,10510.0,515.5,NaN
2,200,10513.0,516.5,NaN
3,300,10517.5,521.5,NaN
4,400,10509.5,512.5,NaN


In [24]:
row = merged.iloc[0]
S = row['mid_price_underlying']
price = row['mid_price_option']
print(f"Option price: {price}, Underlying: {S}, Intrinsic: {max(S - K * exp(-r*T), 0)}")


Option price: 505.5, Underlying: 10503.0, Intrinsic: 516.6892517964407


In [25]:
def objective(sigma):
    val = bs_price(S, K, T, r, sigma, option_type) - price
    print(f"Trying sigma={sigma:.4f}, result={val:.4f}")
    return val


In [26]:
merged['simple_iv'] = np.sqrt(2 * abs(np.log(S / K)) / T)


In [27]:
def safe_iv(row):
    try:
        if any(pd.isnull([row['mid_price_option'], row['mid_price_underlying']])):
            return float('nan')
        if row['mid_price_option'] <= 0 or row['mid_price_underlying'] <= 0:
            return float('nan')
        return implied_volatility(row['mid_price_option'], row['mid_price_underlying'], K, T, r)
    except:
        return float('nan')

merged['implied_vol'] = merged.apply(safe_iv, axis=1)


In [28]:
merged[['timestamp', 'mid_price_underlying', 'mid_price_option', 'implied_vol']].head()


,timestamp,mid_price_underlying,mid_price_option,implied_vol
0,0,10503.0,505.5,NaN
1,100,10510.0,515.5,NaN
2,200,10513.0,516.5,NaN
3,300,10517.5,521.5,NaN
4,400,10509.5,512.5,NaN
